In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wfdb
import os
import csv

In [2]:
records_per_class = 1000

final_labels = pd.read_csv('final_labels.csv')


final_labels = final_labels.dropna()

# Reset index



final_labels = final_labels.reset_index(drop=True)

# Fiter for 1000 record of each class
final_labels = final_labels.groupby('class').head(
    records_per_class).reset_index(drop=True)

# Shuffle the data
final_labels = final_labels.sample(frac=1).reset_index(drop=True)

final_labels

,path,class
0,records500/02000/02433_hr,CD
1,records500/00000/00967_hr,NORM
2,records500/03000/03844_hr,STTC
3,records500/01000/01066_hr,CD
4,records500/00000/00426_hr,NORM
...,...,...
4995,records500/00000/00256_hr,CD
4996,records500/01000/01163_hr,NORM
4997,records500/00000/00642_hr,NORM
4998,records500/00000/00021_hr,NORM


In [12]:
# Function to process a single record
def process_record(record_path, class_label, output_dir):
    file_number = os.path.basename(record_path).split('_')[0]

    # Go one dir above and go data folder with / separator
    record_path = os.path.join(os.path.dirname(
        os.getcwd()), 'data', record_path)

    # Remove the extension
    hea_file = record_path.split('.')[0]

    signals, fields = wfdb.rdsamp(hea_file)
    lead_names = fields.get(
        'sig_name', [f'Lead {i+1}' for i in range(len(signals[0]))])

    csv_file = os.path.join(output_dir, f'{file_number}_{
                            class_label}.csv')  # Include class

    with open(csv_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Time'] + lead_names)
        for time, data_row in zip(range(len(signals)), signals):
            csv_writer.writerow([time] + list(data_row))


# Parameters
output_dir = 'csv_data'
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Process each record
for index, row in final_labels.iterrows():
    process_record(row['path'], row['class'], output_dir)

print('Files saved in csv_data folder.')

Files saved in csv_data folder.
